In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')


from dietrx import db
from dietrx.models import *

import pandas as pd

ModuleNotFoundError: No module named 'pandas'

# Load food

In [2]:
df = pd.read_csv('../data/version 1/food-lexicon-automated.tsv', sep='\t', encoding='utf-8')

df.head()

,Common Names,Food-name,Food ID,Food-category,Scientific Name,TAX ID
0,alexanders; horse parsley,Alexanders,Plant ID:40962,Miscellaneous,Smyrnium olusatrum,40962.0
1,NaN,Scrophularia umbrosa,Plant ID:942083,Miscellaneous,Scrophularia umbrosa,942083.0
2,dotted blazing star,Dotted blazing star,Plant ID:344074,Miscellaneous,Liatris punctata,344074.0
3,plymouth pear,Plymouth pear,Plant ID:761867,Miscellaneous,Pyrus cordata,761867.0
4,NaN,Rhododendron kaempferi,Plant ID:49166,Miscellaneous,Rhododendron kaempferi,49166.0


In [3]:
df = df.where((pd.notnull(df)), None)

df.rename(columns={
    'Common Names': 'common_names',
    'Food-name': 'display_name',
    'Food ID': 'food_id',
    'Food-category': 'food_category',
    'Scientific Name': 'scientific_name',
    'TAX ID':'tax_id',
},inplace=True)


data_list= list(df.T.to_dict().values())

In [4]:
for item in data_list:
    db.session.add(Food(**item))
    
db.session.commit()

# Load Disease

In [5]:
df = pd.read_csv('../data/version 1/disease-lexicon.tsv', sep='\t', encoding='utf-8', index_col=0)

df.head()

,Disease ID,Disease-name,Disease-synonyms,Disease-category
0,MESH:D005236,Favism,"Favisms|FAVISM, SUSCEPTIBILITY TO",Blood disease|Genetic disease (inborn)|Chemica...
1,MESH:D003715,Dengue,Breakbone Fever|Break Bone Fever|Break-Bone Fe...,Viral disease
2,MESH:D034801,Mastocytoma,Benign Mastocytoma|Extracutaneous Mastocytoma|...,Cancer|Skin disease
3,MESH:D002169,Campylobacter Infections,Campylobacter Infection|Campylobacterioses|Cam...,Bacterial infection or mycosis
4,MESH:D045888,Ganglion Cysts,"Cyst, Ganglion|Cyst, Ganglionic|Cyst, Myxoid|C...",Cancer|Connective tissue disease


In [6]:
df = df.where((pd.notnull(df)), None)

df.rename(columns={
    'Disease ID': 'disease_id',
    'Disease-name': 'disease_name',
    'Disease-synonyms': 'disease_synonyms',
    'Disease-category': 'disease_category',
},inplace=True)


data_list= list(df.T.to_dict().values())

In [7]:
for item in data_list:
    db.session.add(Disease(**item))
    
db.session.commit()

# Load Gene

In [25]:
df = pd.read_csv('../data/version 1/gene-lexicon.tsv', sep='\t', encoding='utf-8')

df.head()

,Gene ID
0,COX14
1,CLCN3
2,KIAA1549
3,RHPN1
4,MSX2


In [9]:
df.rename(columns={
    'Gene ID': 'gene_id',
},inplace=True)


data_list= list(df.T.to_dict().values())

In [10]:
for item in data_list:
    db.session.add(Gene(**item))
    
db.session.commit()

# Load References

In [11]:
df = pd.read_csv('../data/version 1/publication-records.tsv', sep='\t', encoding='utf-8', index_col=0)

df.head()

,Authors,Date,Journal Name,Journal Name Abbrv.,PMID,Publication Type,Title
0,Elkiran T|Harputluoglu H|Yasar U|Babaoglu MO|D...,2007 Jan-Feb,Methods and findings in experimental and clini...,Methods Find Exp Clin Pharmacol,17344941,Clinical Trial|Journal Article,Differential alteration of drug-metabolizing e...
1,Severins N|Mensink RP|Plat J,2015 Feb,"Nutrition, metabolism, and cardiovascular dise...",Nutr Metab Cardiovasc Dis,25456153,Journal Article|Randomized Controlled Trial,Effects of lutein-enriched egg yolk in butterm...
2,Li C|Zuo C|Deng G|Kuang R|Yang Q|Hu C|Sheng O|...,2013,PloS one,PLoS One,23922960,Journal Article,Contamination of bananas with beauvericin and ...
3,Güneser S|Atici A|Cengizler I|Alparslan N,1996 May-Jun,Allergologia et immunopathologia,Allergol Immunopathol (Madr),8766742,Journal Article,Inhalant allergens: as a cause of respiratory ...
4,Brvar M|Bunc M,2009 Sep 9,Cases journal,Cases J,19918387,Journal Article,High-degree atrioventricular block in acute et...


In [12]:
df = df.where((pd.notnull(df)), None)

df.rename(columns={
    'Authors': 'authors',
    'Date': 'date',
    'Journal Name': 'journal_name',
    'Journal Name Abbrv.': 'journal_name_abbr',
    'PMID': 'pmid',
    'Publication Type':'publication_type',
    'Title':'title',
},inplace=True)


data_list= list(df.T.to_dict().values())

In [13]:
for item in data_list:
    db.session.add(References(**item))
    
db.session.commit()

# Load Food Disease

In [14]:
df = pd.read_csv('../data/version 1/food-disease.tsv', sep='\t', encoding='utf-8', index_col=0)


df = df.where((pd.notnull(df)), None)


df['Association'].replace(to_replace='Negative Association', value='negative', inplace=True)
df['Association'].replace(to_replace='Positive Association', value='positive', inplace=True)
df['Association'].replace(to_replace='No Association', value='no', inplace=True)
df['Association'].replace(to_replace='Plant-Disease Association', value='plant disease', inplace=True)

df = df[df.Association != 'no']
df = df[df.Association != 'plant disease']

df.rename(columns={
    'PMID': 'pmid',
    'Food ID': 'food_id',
    'Disease ID': 'disease_id',
    'Association': 'association',
},inplace=True)


data_list= list(df.T.to_dict().values())


df.head()

/home/neelansh/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:881: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  "columns will be omitted.", UserWarning)


,pmid,food_id,disease_id,association
0,7398283,Plant ID:4045,MESH:D004487,negative
1,7398283,Plant ID:4045,MESH:D014581,negative
2,7398283,Plant ID:4043,MESH:D004487,negative
3,7398283,Plant ID:4043,MESH:D014581,negative
5,24268374,Plant ID:3483,MESH:D021081,negative


In [15]:
for item in data_list:
    db.session.add(Food_disease(**item))
    
db.session.commit()

# Load Disease Gene

In [18]:
df = pd.read_csv('../data/version 1/disease-gene.tsv', sep='\t', encoding='utf-8', index_col=0)

df.drop_duplicates(inplace=True)

df.rename(columns={
    'Gene Symbol': 'gene_id',
    'Disease ID': 'disease_id',
    'Reference': 'reference',
},inplace=True)

df.head()

data_list= list(df.T.to_dict().values())

In [22]:
for item in data_list:
    db.session.add(Disease_gene(**item))
    
db.session.commit()

# Load Food Gene

In [3]:
df = pd.read_csv('../data/version 1/food-gene.tsv', sep='\t', encoding='utf-8')

df.rename(columns={
    'Food ID': 'food_id',
    'Gene ID': 'gene_id',
    'Disease-categories': 'disease_categories',
},inplace=True)

data_list= list(df.T.to_dict().values())

df.head()

,food_id,gene_id,disease_categories
0,AlcoholicBev ID:1,RFH1,Urogenital disease (male)|Congenital abnormali...
1,AlcoholicBev ID:16,A2M,Mental disorder
2,AlcoholicBev ID:16,AAAS,Urogenital disease (male)|Cancer
3,AlcoholicBev ID:16,AANAT,Mental disorder
4,AlcoholicBev ID:16,AASS,Blood disease


In [4]:
for i, item in enumerate(data_list):
    if i%10000 != 0:
        db.session.add(Food_gene(**item))
    else:
        db.session.add(Food_gene(**item))
        db.session.commit()
        
db.session.commit()

In [2]:
# for item in Food_gene.query.all():
#     db.session.delete(item)
    
# db.session.commit()

In [31]:
# db.session.rollback()